In [2]:
### This is about simulating a great number of times the Velib trafic in Rouen
import Distributions
using Distributions
srand(123);

In [3]:
#General Data
T = 1000 #horizon
n = 5 #number of stations : 3-7
dist_matrix = [0 3 5 7 7;
               2 0 2 5 5;
               4 2 0 3 3;
               8 6 4 0 2;
               7 7 5 2 0]; #distance between stations
terminal_number = [24 20 20 15 20]; #number of terminals at each station
routing_matrix = [0 0.2 0.3 0.2 0.3;
                  0.2 0 0.3 0.2 0.3;
                  0.2 0.25 0 0.25 0.3;
                  0.15 0.2 0.3 0 0.35;
                  0.2 0.25 0.35 0.2 0]; #routing matrix ?
leaving_per_hour = [2.8 3.7 5.5 3.5 4.6]; #mean number of bikes leaving each station per hour
alpha = 0.1;
N_bikes = convert(Integer, 91); #total number of bikes
N_stations = convert(Integer, 5); #number of stations

In [4]:
#Initial data
X0 = [20 1 0 0 0;
     1 15 1 0 0;
     0 1 17 1 0;
     0 0 1 13 1;
     0 0 0 1 18];

In [5]:
function get_event(v, P)
    i0 = 1; j0 = 1; s = 0;
    for i in range(1, N_stations)
        for j in range(1, N_stations)
            s = s + P[(i-1)*N_stations + j];
            if v > s
                i0 = i; j0 = j;
            else
                return [i0, j0];
            end
        end
    end
    return [i0, j0]
end

get_event (generic function with 1 method)

In [6]:
function simulate(T)
    #define distributions
    c = Uniform();
    route = [Categorical(routing_matrix[i, :]) for i in range(1, N_stations)]; #TO BE CHANGED
    #time
    t = 0;
    #state
    X = X0;
    #number of problems
    result = 0;
    while (t < T)
        #********************************************************************
        #Get the parameter of the stay time Exponential 
        #get also the vector with probabilities of each event p
        lambda = 0;
        p = zeros(N_stations*N_stations);
        for i in range(1, N_stations)
            for j in range(1, N_stations)
                if i != j
                    lambda = lambda + X[i,j]*dist_matrix[i,j];
                    p[N_stations*(i-1) + j] = X[i,j]*dist_matrix[i,j];
                else
                    lambda = lambda + alpha * leaving_per_hour[i];
                    p[N_stations*(i-1) + j] = alpha * leaving_per_hour[i];
                end
            end
        end
        p = p./lambda;
        
        #*****************************************************************
        #Draw the time to stay
        exp = Exponential(lambda);
        dt = rand(exp);
        t = t + dt;
        
        #*****************************************************************
        #draw event and realise it
        cat = rand(c);
        A = get_event(cat, p); i = A[1]; j = A[2];
        #if i == j it is a departure. Else it is an arrival.
        if i == j
            #if there is no bike available
            if X[i,j] == 0
                result = result + 1;
            else
                X[i,j] = X[i,j] - 1;
                destination = rand(route[i]);
                X[i, destination] = X[i, destination] + 1;
            end
            println("departure ", t);
        else #it is an arrival
            #if no terminal is available
            if X[j,j] == terminal_number[j]
                result = result + 1;
            else
                X[i,j] = X[i,j] - 1;
                X[j,j] = X[j,j] + 1;
            end
            println("arrival ", t);
        end
    end
    return result
end

simulate (generic function with 1 method)

In [7]:
simulate(1000)

arrival 2.79289154812082
departure 11.449844564942861
arrival 29.04054731610867
arrival 32.110417394289094
departure 33.00400120664936
arrival 49.50547581164237
arrival 54.65008272196967
departure 57.43907151698181
arrival 69.57252958696289
arrival 70.85999915274348
departure 72.24849704100242
departure 93.24410119029281
departure 122.98572636735594
departure 124.92875232281
departure 126.06616642452772
arrival 130.2762578407134
departure 145.1865964591952
departure 168.9515440276628
departure 171.7246270196958
departure 216.64306210217273
departure 257.1234450661543
departure 267.39234219777694
arrival 297.71521386093497
departure 310.52264167094813
arrival 311.511179872061
departure 350.23594711787814
arrival 368.0658685135713
departure 387.37997541853116
arrival 578.5406755729953
arrival 610.0073521545287
arrival 628.5269560914664
arrival 758.3229757548436
departure 809.2094468561222
arrival 825.0845148594511
arrival 836.8959799466895
arrival 863.1597108057335
departure 915.35598026

8